In [1]:
import os
import sys
sys.path.append('../input/pytorch-images-seresnet')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
import albumentations
from albumentations import *
from albumentations.pytorch import ToTensorV2


import timm

from torch.cuda.amp import autocast, GradScaler

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16
#Please note - train path being called in place of test
TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test'
#Please note - train dataset being called in place of test
test = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')
#test = test[0:100]

class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}.jpg'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image
def get_transforms(image_size=640):
        return Compose([
            Resize(image_size, image_size),
            Normalize(),
            ToTensorV2(),
        ])
    
    
class RANZCRResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d', out_dim=11, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, out_dim)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output    
    
def inference_FC(models, test_loader, device):
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    FC_layer = []
    FC_layer_TTA = []
    for i, (images) in tk0:
        images = images.to(device)
        FC = []
        FC_TTA = []
        for model in models:
            with torch.no_grad():
                y_preds1 = model(images)
                y_preds2 = model(images.flip(-1))
            FC.append(y_preds1.to('cpu').numpy())
            FC_TTA.append(y_preds2.to('cpu').numpy())
        FC_layer.append(FC)
        FC_layer_TTA.append(FC_TTA)
    FC_layer = np.hstack(FC_layer)
    FC_layer_TTA = np.hstack(FC_layer_TTA)
    return FC_layer, FC_layer_TTA
    
#===================================================================================================================================================================    
models200D_UW1 = []
model = RANZCRResNet200D()
model.load_state_dict(torch.load("../input/resnet200d-baseline-benchmark-public/resnet200d_fold0_cv953.pth", map_location='cuda:0'))
x=list(model.children())
x1= x[:-1]
model = torch.nn.Sequential(*x1)
model.eval()
model.to(device)
models200D_UW1.append(model)

models200D_UW2 = []
model = RANZCRResNet200D()
model.load_state_dict(torch.load("../input/resnet200d-baseline-benchmark-public/resnet200d_fold1_cv955.pth", map_location='cuda:0'))
x=list(model.children())
x1= x[:-1]
model = torch.nn.Sequential(*x1)
model.eval()
model.to(device)
models200D_UW2.append(model)

models200D_UW3 = []
model = RANZCRResNet200D()
model.load_state_dict(torch.load("../input/resnet200d-baseline-benchmark-public/resnet200d_fold2_cv955.pth", map_location='cuda:0'))
x=list(model.children())
x1= x[:-1]
model = torch.nn.Sequential(*x1)
model.eval()
model.to(device)
models200D_UW3.append(model)

models200D_UW4 = []
model = RANZCRResNet200D()
model.load_state_dict(torch.load("../input/resnet200d-baseline-benchmark-public/resnet200d_fold3_cv957.pth", map_location='cuda:0'))
x=list(model.children())
x1= x[:-1]
model = torch.nn.Sequential(*x1)
model.eval()
model.to(device)
models200D_UW4.append(model)

models200D_UW5 = []
model = RANZCRResNet200D()
model.load_state_dict(torch.load("../input/resnet200d-baseline-benchmark-public/resnet200d_fold4_cv954.pth", map_location='cuda:0'))
x=list(model.children())
x1= x[:-1]
model = torch.nn.Sequential(*x1)
model.eval()
model.to(device)
models200D_UW5.append(model)


test_dataset_512 = TestDataset(test, transform=get_transforms(image_size=512))
test_loader_512 = DataLoader(test_dataset_512, batch_size=BATCH_SIZE, shuffle=False, num_workers=4 , pin_memory=True)

predictions200d_UW1, predictions200d_UW1_TTA = inference_FC(models200D_UW1, test_loader_512, device)
predictions200d_UW1 = np.squeeze(np.squeeze(np.asarray(predictions200d_UW1)))
print(predictions200d_UW1.shape); np.save('features_FC_model_UW1.npy', predictions200d_UW1); del predictions200d_UW1
predictions200d_UW1_TTA = np.squeeze(np.squeeze(np.asarray(predictions200d_UW1_TTA)))
print(predictions200d_UW1_TTA.shape); np.save('features_FC_model_UW1_TTA.npy', predictions200d_UW1_TTA); del predictions200d_UW1_TTA

predictions200d_UW2, predictions200d_UW2_TTA = inference_FC(models200D_UW2, test_loader_512, device)
predictions200d_UW2 = np.squeeze(np.squeeze(np.asarray(predictions200d_UW2)))
print(predictions200d_UW2.shape); np.save('features_FC_model_UW2.npy', predictions200d_UW2); del predictions200d_UW2
predictions200d_UW2_TTA = np.squeeze(np.squeeze(np.asarray(predictions200d_UW2_TTA)))
print(predictions200d_UW2_TTA.shape); np.save('features_FC_model_UW2_TTA.npy', predictions200d_UW2_TTA); del predictions200d_UW2_TTA

predictions200d_UW3, predictions200d_UW3_TTA = inference_FC(models200D_UW3, test_loader_512, device)
predictions200d_UW3 = np.squeeze(np.squeeze(np.asarray(predictions200d_UW3)))
print(predictions200d_UW3.shape); np.save('features_FC_model_UW3.npy', predictions200d_UW3); del predictions200d_UW3
predictions200d_UW3_TTA = np.squeeze(np.squeeze(np.asarray(predictions200d_UW3_TTA)))
print(predictions200d_UW3_TTA.shape); np.save('features_FC_model_UW3_TTA.npy', predictions200d_UW3_TTA); del predictions200d_UW3_TTA

predictions200d_UW4, predictions200d_UW4_TTA = inference_FC(models200D_UW4, test_loader_512, device)
predictions200d_UW4 = np.squeeze(np.squeeze(np.asarray(predictions200d_UW4)))
print(predictions200d_UW4.shape); np.save('features_FC_model_UW4.npy', predictions200d_UW4); del predictions200d_UW4
predictions200d_UW4_TTA = np.squeeze(np.squeeze(np.asarray(predictions200d_UW4_TTA)))
print(predictions200d_UW4_TTA.shape); np.save('features_FC_model_UW4_TTA.npy', predictions200d_UW4_TTA); del predictions200d_UW4_TTA

predictions200d_UW5, predictions200d_UW5_TTA = inference_FC(models200D_UW5, test_loader_512, device)
predictions200d_UW5 = np.squeeze(np.squeeze(np.asarray(predictions200d_UW5)))
print(predictions200d_UW5.shape); np.save('features_FC_model_UW5.npy', predictions200d_UW5); del predictions200d_UW5
predictions200d_UW5_TTA = np.squeeze(np.squeeze(np.asarray(predictions200d_UW5_TTA)))
print(predictions200d_UW5_TTA.shape); np.save('features_FC_model_UW5_TTA.npy', predictions200d_UW5_TTA); del predictions200d_UW5_TTA

del test_dataset_512, test_loader_512

del models200D_UW1, models200D_UW2, models200D_UW3, models200D_UW4, models200D_UW5
#=========================================================================================================================================================


FC_RESNET_UW1 = np.load('./features_FC_model_UW1.npy')
FC_RESNET_UW1_TTA = np.load('./features_FC_model_UW1_TTA.npy')
FC_RESNET_UW2 = np.load('./features_FC_model_UW2.npy')
FC_RESNET_UW2_TTA = np.load('./features_FC_model_UW2_TTA.npy')
FC_RESNET_UW3 = np.load('./features_FC_model_UW3.npy')
FC_RESNET_UW3_TTA = np.load('./features_FC_model_UW3_TTA.npy')
FC_RESNET_UW4 = np.load('./features_FC_model_UW4.npy')
FC_RESNET_UW4_TTA = np.load('./features_FC_model_UW4_TTA.npy')
FC_RESNET_UW5 = np.load('./features_FC_model_UW5.npy')
FC_RESNET_UW5_TTA = np.load('./features_FC_model_UW5_TTA.npy')

from sklearn.preprocessing import minmax_scale
FC_RESNET_UW1 = minmax_scale(FC_RESNET_UW1, feature_range=(0,1))
FC_RESNET_UW1_TTA = minmax_scale(FC_RESNET_UW1_TTA, feature_range=(0,1))
FC_RESNET_UW2 = minmax_scale(FC_RESNET_UW2, feature_range=(0,1))
FC_RESNET_UW2_TTA = minmax_scale(FC_RESNET_UW2_TTA, feature_range=(0,1))
FC_RESNET_UW3 = minmax_scale(FC_RESNET_UW3, feature_range=(0,1))
FC_RESNET_UW3_TTA = minmax_scale(FC_RESNET_UW3_TTA, feature_range=(0,1))
FC_RESNET_UW4 = minmax_scale(FC_RESNET_UW4, feature_range=(0,1))
FC_RESNET_UW4_TTA = minmax_scale(FC_RESNET_UW4_TTA, feature_range=(0,1))
FC_RESNET_UW5 = minmax_scale(FC_RESNET_UW5, feature_range=(0,1))
FC_RESNET_UW5_TTA = minmax_scale(FC_RESNET_UW5_TTA, feature_range=(0,1))

#FC_ALL1 = np.hstack([FC_RESNET,FC_RESNET_TTA,FC_SERESNET,FC_SERESNET_TTA,FC_EFFNET,FC_EFFNET_TTA,FC_DENSNET,FC_DENSNET_TTA])
FC_ALL2 = np.hstack([FC_RESNET_UW1,FC_RESNET_UW1_TTA,FC_RESNET_UW2,FC_RESNET_UW2_TTA,FC_RESNET_UW3,FC_RESNET_UW3_TTA,FC_RESNET_UW4,FC_RESNET_UW4_TTA,FC_RESNET_UW5,FC_RESNET_UW5_TTA])
#FC_ALL1 = pd.DataFrame(FC_ALL1)
FC_ALL2 = pd.DataFrame(FC_ALL2)

#del FC_RESNET,FC_RESNET_TTA,FC_SERESNET,FC_SERESNET_TTA,FC_EFFNET,FC_EFFNET_TTA,FC_DENSNET,FC_DENSNET_TTA
del FC_RESNET_UW1,FC_RESNET_UW1_TTA,FC_RESNET_UW2,FC_RESNET_UW2_TTA,FC_RESNET_UW3,FC_RESNET_UW3_TTA,FC_RESNET_UW4,FC_RESNET_UW4_TTA,FC_RESNET_UW5,FC_RESNET_UW5_TTA

targets = [
    "ETT - Abnormal",
    "ETT - Borderline",
    "ETT - Normal",
    "NGT - Abnormal",
    "NGT - Borderline",
    "NGT - Incompletely Imaged",
    "NGT - Normal",
    "CVC - Abnormal",
    "CVC - Borderline",
    "CVC - Normal",
    "Swan Ganz Catheter Present",
]

from sklearn import metrics
from keras.models import load_model

n_fold = 5
result = np.zeros([FC_ALL2.shape[0],11])

for zzz in range(n_fold):
    #filepath = '../input/ranzcr-nn-level2-mar16/FC1' + '_fold_' + str(zzz) + '.h5'
    #model = load_model(filepath, compile = True)
    #preds1 = model.predict(FC_ALL1.values)

    filepath = '../input/ranzcr-nn-level2-mar16/FC2' + '_fold_' + str(zzz) + '.h5'
    model = load_model(filepath, compile = True)
    preds2 = model.predict(FC_ALL2.values)
    
    preds = (preds2 + preds2)/2.0
    result += preds/n_fold
    
target_cols = test.iloc[:, 1:12].columns.tolist()
test[target_cols] = result
test[['StudyInstanceUID'] + target_cols].to_csv('submission.csv', index=False)
test.head()

#=========================================================================================================================================

  0%|          | 0/224 [00:00<?, ?it/s]

(3582, 2048)
(3582, 2048)


  0%|          | 0/224 [00:00<?, ?it/s]

(3582, 2048)
(3582, 2048)


  0%|          | 0/224 [00:00<?, ?it/s]

...

  0%|          | 0/224 [00:00<?, ?it/s]

(3582, 2048)
(3582, 2048)


  0%|          | 0/224 [00:00<?, ?it/s]

(3582, 2048)
(3582, 2048)


,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0.028865,0.428791,0.482511,0.010203,0.009173,0.017337,0.987773,0.045632,0.057336,0.872689,0.986408
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,0.000237,0.000261,0.000670,0.000327,0.000381,0.000316,0.000286,0.003978,0.001107,0.999450,0.000162
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,0.000504,0.000433,0.000426,0.000666,0.000620,0.000507,0.000421,0.004108,0.640104,0.490784,0.000400
3,1.2.826.0.1.3680043.8.498.84994474380235968109...,0.001842,0.025429,0.015028,0.016412,0.010628,0.998132,0.004664,0.023463,0.016698,0.989687,0.004946
4,1.2.826.0.1.3680043.8.498.35798987793805669662...,0.000522,0.000563,0.000713,0.000867,0.000914,0.000607,0.000637,0.010837,0.173221,0.927719,0.000392
